In [4]:
import tensorflow as tf
import tensorflow.keras as keras
from keras.losses import mse, binary_crossentropy
import tensorflow.keras.backend as K
from datetime import datetime
from tensorflow.keras import layers
from os.path import join
import pandas as pd
import numpy as np
from config import processed_data_path

### Data


In [6]:
def data(variant, category, random_seed=42):
    # variant -> 'VLQ_HG', 'VLQ_SEM_HG', 'bkg', 'FCNC'
    # category -> train, validation, test
    
    # TODO: Improve efficiency/handle names
    
    # Check if variant is valid
    assert variant in {'VLQ_HG', 'VLQ_SEM_HG', 'bkg', 'FCNC'}, "Invalid variant!"

    # With specified variant, get data
    file = join(processed_data_path, variant+".csv")
    data = pd.read_csv(file, index_col=0)

    # Shuffle the dataframe
    data = data.sample(frac=1, random_state=random_seed).reset_index(drop=True)

    # This will equally devide the dataset into 
    # train, validation and test
    train, validation, test = np.split(data.sample(frac=1), [int(len(data)*(1/3)), int(len(data)*(2/3))])
    
    if category == "train":
        data = train
    elif category == "validation":
        data = validation
    elif category == "test":
        data = test
    
    # This data we want on a seperate variable
    weights = data["weights"].to_numpy(dtype=np.float16)
    name = data["name"]

    data.drop(["name", "weights"], axis=1, inplace=True)

    data = data.to_numpy(dtype=np.float16)
    return data, weights, name

In [8]:
# Data
x_train, x_train_weights, _ = data(category='train',
                                    variant='bkg')
x_val, _, _ = data(category='validation',
                                variant='bkg')


In [10]:
class Reparametrize(keras.layers.Layer):
    def call(self, inputs):
        mean, log_var = inputs
        return K.random_normal(tf.shape(log_var)) * K.exp(log_var / 2) + mean

In [12]:
hidden_size = 2

## Encoder
inputs = layers.Input(shape=(69,))
z = layers.Dense(128)(inputs)
z = layers.LeakyReLU()(z)

mean = layers.Dense(hidden_size)(z)
log_var = layers.Dense(hidden_size)(z)                
latent = Reparametrize()([mean, log_var])

encoder = keras.Model(inputs=[inputs], outputs=[mean, log_var, latent], name="encoder")
encoder.summary()

Model: "encoder"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_3 (InputLayer)            [(None, 69)]         0                                            
__________________________________________________________________________________________________
dense_5 (Dense)                 (None, 128)          8960        input_3[0][0]                    
__________________________________________________________________________________________________
leaky_re_lu_3 (LeakyReLU)       (None, 128)          0           dense_5[0][0]                    
__________________________________________________________________________________________________
dense_6 (Dense)                 (None, 2)            258         leaky_re_lu_3[0][0]              
____________________________________________________________________________________________

In [14]:
## Decoder 
decoder_inputs = layers.Input(shape=[hidden_size])
z = layers.Dense(128)(decoder_inputs)
z = layers.LeakyReLU()(z)
z = layers.Dense(69)(z)
outputs = layers.LeakyReLU()(z)

decoder = keras.Model(inputs=[decoder_inputs], outputs=[outputs], name="decoder")
decoder.summary()

## Joining everything
#_, _, latent = encoder(inputs)
#reconstruction = decoder(latent)
#model = keras.Model(inputs=[inputs], outputs=[reconstruction])

Model: "decoder"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_4 (InputLayer)         [(None, 2)]               0         
_________________________________________________________________
dense_8 (Dense)              (None, 128)               384       
_________________________________________________________________
leaky_re_lu_4 (LeakyReLU)    (None, 128)               0         
_________________________________________________________________
dense_9 (Dense)              (None, 69)                8901      
_________________________________________________________________
leaky_re_lu_5 (LeakyReLU)    (None, 69)                0         
Total params: 9,285
Trainable params: 9,285
Non-trainable params: 0
_________________________________________________________________


In [16]:
class VAE(keras.Model):
    def __init__(self, encoder, decoder, **kwargs):
        super(VAE, self).__init__(**kwargs)
        self.encoder = encoder
        self.decoder = decoder
        # Train
        self.total_train_loss_tracker = keras.metrics.Mean(name="total_train_loss")
        self.recon_train_loss_tracker = keras.metrics.Mean(name="recon_train_loss")
        self.kl_train_loss_tracker = keras.metrics.Mean(name="kl_train_loss")
        # Val
        self.total_val_loss_tracker = keras.metrics.Mean(name="total_val_loss")
        self.recon_val_loss_tracker = keras.metrics.Mean(name="recon_val_loss")
        self.kl_val_loss_tracker = keras.metrics.Mean(name="kl_val_loss")

    @property
    def metrics(self):
        return [
            self.total_train_loss_tracker,
            self.recon_train_loss_tracker,
            self.kl_train_loss_tracker,
            self.total_val_loss_tracker,
            self.recon_val_loss_tracker,
            self.kl_val_loss_tracker
        ]
        
    def train_step(self, data):
        with tf.GradientTape() as tape:
            x, y, weights = data

            z_mean, z_log_var, z = self.encoder(x)
            reconstruction = self.decoder(z)

            ## Loss
            # reconstruction
            recon_loss = binary_crossentropy(x, reconstruction) # Shape = BATCH_SIZE
            # Weights on recon loss
            recon_train_loss = (weights * recon_loss) / K.sum(weights)
            recon_loss = K.mean(recon_loss, axis = 0)

            # KL
            kl_loss = -0.5 * (1 + z_log_var - K.square(z_mean) - K.exp(z_log_var))
            kl_loss = K.mean(K.sum(kl_loss, axis=1), axis=0)
            # Weights on KL Loss
            kl_loss = (weights * kl_loss) / K.sum(weights)
            kl_loss = K.mean(kl_loss, axis = 0)

            # Total
            total_loss = recon_loss + kl_loss

        # Step
        grads = tape.gradient(total_loss, self.trainable_weights)
        self.optimizer.apply_gradients(zip(grads, self.trainable_weights))
        
        # Log
        self.total_train_loss_tracker.update_state(total_loss)
        self.recon_train_loss_tracker.update_state(recon_loss)
        self.kl_train_loss_tracker.update_state(kl_loss)

        return {
            "total_train_loss": self.total_train_loss_tracker.result(),
            "recon_train_loss": self.recon_train_loss_tracker.result(),
            "kl_train_loss": self.kl_train_loss_tracker.result(),
        }

    def validation_step(self, data):
        # No tape, we don't need gradients
        x, y = data

        print("1", type(x), type(y))

        z_mean, z_log_var, z = self.encoder(x)
        reconstruction = self.decoder(z)

        ## Loss
        # reconstruction
        recon_loss = binary_crossentropy(x, reconstruction) # Shape = BATCH_SIZE

        # KL
        kl_loss = -0.5 * (1 + z_log_var - K.square(z_mean) - K.exp(z_log_var))
        kl_loss = K.mean(K.sum(kl_loss, axis=1), axis=0)

        # Total
        total_loss = recon_loss + kl_loss


        loss = recon_loss*self.alpha + kl_loss

        # Log
        self.total_val_loss_tracker.update_state(total_loss)
        self.recon_val_loss_tracker.update_state(recon_loss)
        self.kl_val_loss_tracker.update_state(kl_loss)

        return {
            "total_val_loss": self.total_val_loss_tracker.result(),
            "recon_val_loss": self.recon_val_loss_tracker.result(),
            "kl_val_loss": self.kl_val_loss_tracker.result(),
        }

    def test_step(self, data):
        # No tape, we don't need gradients
        x, y = data

        z_mean, z_log_var, z = self.encoder(x)
        reconstruction = self.decoder(z)
        return reconstruction, z_mean, z_log_var, z # z is the latent vector




### Train

In [ ]:
## Callbacks
# Model name
name = str(datetime.now().strftime("%d_%m_%Y__%H_%M_%S"))

# Tensorboard
TB = keras.callbacks.TensorBoard(log_dir=join("logs", name), write_images=True)

# Early Stopping
ES = keras.callbacks.EarlyStopping(monitor="total_val_loss", patience=30, verbose=2, mode="min")

# Model Checkpoint
MC = keras.callbacks.ModelCheckpoint(filepath=join("models_tf", name), save_best_only=True, monitor="total_val_loss", mode="min")

In [15]:
# Compile
vae = VAE(encoder, decoder)
vae.compile(optimizer=keras.optimizers.Adam())

In [17]:
# Fit
history = vae.fit(
    # Train
    x=x_train, 
    y=x_train,
    sample_weight=x_train_weights,

    # Validation
    validation_data=(x_val,x_val),

    # Hyper-parameters    
    epochs=30, 
    batch_size=4048,
    callbacks=[TB, ES, MC])

Epoch 1/30
90/92 [============================>.] - ETA: 0s - total_train_loss: -0.4637 - recon_train_loss: -0.4832 - kl_train_loss: 0.0194

AttributeError: 'tuple' object has no attribute 'items'

### Ploting

In [19]:
def plot_label_clusters(vae, data, labels):
    # display a 2D plot of the digit classes in the latent space
    z_mean, _, _ = vae.encoder.predict(data)
    plt.figure(figsize=(12, 10))
    plt.scatter(z_mean[:, 0], z_mean[:, 1], c=labels)
    plt.colorbar()
    plt.xlabel("z[0]")
    plt.ylabel("z[1]")
    plt.show()


x_test, _, x_test_names = data(category='test',
                                    variant='bkg')

plot_label_clusters(vae, x_train, y_train)

NameError: name 'y_train' is not defined